<a href="https://colab.research.google.com/github/vrindavnair-ai/Collab_ML_DL/blob/main/transformer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://devocean.sk.com/blog/techBoardDetail.do?ID=165703 바탕으로 진행


#기본 환경설정


In [ ]:
!pip3 install -q --upgrade transformers
!pip3 install -q --upgrade datasets
!pip3 install -q --upgrade bitsandbytes
!pip3 install -q --upgrade peft
!pip3 install -q --upgrade trl
!pip3 install -q --upgrade accelerate
!pip3 install -q --upgrade huggingface_hub
!pip3 install -q --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 19.7 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# Load model directly
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

# 데이터셋 불러오기

In [ ]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("Akshaydevaraboina/workoutplanGenerator")
# 데이터셋을 Pandas DataFrame으로 변환
df1 = pd.DataFrame(ds1['train'])
df1 = df1.drop(columns=['text'])
# CSV 파일로 저장
df1.to_csv('output_file.csv', index=False)


ds2 = load_dataset("CristiD7/Comprehensive_7Day_Workout_Plans")
# 데이터셋을 Pandas DataFrame으로 변환
df2 = pd.DataFrame(ds2['train'])  # 'train'을 사용하거나 다른 스플릿이 있을 경우 변경 가능
# CSV 파일로 저장
df2.to_csv('output_file.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/344 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/34.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/900 [00:00<?, ? examples/s]

workout_dataset.jsonl:   0%|          | 0.00/679k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/900 [00:00<?, ? examples/s]

In [ ]:
# 두 DataFrame을 수직으로 결합 (행 추가)
df_combined = pd.concat([df1, df2], ignore_index=True)

# 결합된 DataFrame을 CSV로 저장
df_combined.to_csv('combined_file.csv', index=False)
df_combined.head()

,Context,Response
0,I am a 39-year-old female with a height of 182...,Monday:\n- Mountain Climbers: 3 sets of 20 rep...
1,I am a 35-year-old male with a height of 161 c...,Monday:\n- Bicep Curls: 3 sets of 12 reps\n- S...
2,I am a 35-year-old female with a height of 154...,Monday:\n- Kettlebell Swings: 3 sets of 6 reps...
3,I am a 58-year-old female with a height of 185...,Monday:\n- Pull-ups: 3 sets of 10 reps\n- Benc...
4,I am a 59-year-old male with a height of 155 c...,Monday:\n- Tricep Dips: 3 sets of 20 reps\n- S...


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# CSV 파일 불러오기
df = df_combined

# 'Context'와 'Response' 열을 'QUESTION'과 'ANSWER'로 이름 변경
df = df.rename(columns={'Context': 'QUESTION', 'Response': 'ANSWER'})

# train/test 데이터셋으로 나누기 (예: 80%는 train, 20%는 test)
train_df, test_df = train_test_split(df, test_size=0.2)

# Pandas DataFrame을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# DatasetDict 출력 (선택 사항)
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['QUESTION', 'ANSWER', '__index_level_0__'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['QUESTION', 'ANSWER', '__index_level_0__'],
        num_rows: 360
    })
})


# 프롬프트 테스트


In [ ]:
# messages 형식으로 프롬프트 생성 함수 정의
def create_messages(example):
    return [
        {"role": "user", "content": example['QUESTION']},
        {"role": "assistant", "content": example['ANSWER']}
    ]

# 데이터셋 변환
dataset_dict['train'] = dataset_dict['train'].map(lambda example: {'messages': create_messages(example)})

# 결과 확인
print(dataset_dict['train'][0]['messages'])


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

[{'content': 'I am a 41-year-old female with a height of 200 cm and a weight of 76 kg. My primary goal is weight loss, and I aim to reach a target weight of 71 kg. I am a advanced at the gym. Can you create a 7-day workout plan for me?', 'role': 'user'}, {'content': 'Monday:\n- Push-ups: 3 sets of 6 reps\n- Squats: 3 sets of 10 reps\n- Mountain Climbers: 3 sets of 10 reps\nTuesday:\n- Push-ups: 3 sets of 12 reps\n- Tricep Dips: 3 sets of 12 reps\n- High Knees: 3 sets of 6 reps\nWednesday:\n- Rest Day\nThursday:\n- Kettlebell Swings: 3 sets of 15 reps\n- Jumping Jacks: 3 sets of 12 reps\n- Sit-ups: 3 sets of 20 reps\nFriday:\n- Deadlifts: 3 sets of 8 reps\n- Bicep Curls: 3 sets of 6 reps\n- Dumbbell Rows: 3 sets of 15 reps\nSaturday:\n- Rest Day\nSunday:\n- Rest Day', 'role': 'assistant'}]


In [ ]:
message = dataset_dict['train'][0]['messages']
print(message)

[{'content': 'I am a 35-year-old male with a height of 161 cm and a weight of 50 kg. My primary goal is muscle gain, and I aim to reach a target weight of 55 kg. I am a beginner at the gym. Can you create a 7-day workout plan for me?', 'role': 'user'}, {'content': 'Monday:\n- Bicep Curls: 3 sets of 12 reps\n- Squats: 3 sets of 20 reps\n- Running: 3 sets of 8 reps\nTuesday:\n- Russian Twists: 3 sets of 12 reps\n- Shoulder Press: 3 sets of 12 reps\n- Jump Rope: 3 sets of 8 reps\nWednesday:\n- Rest Day\nThursday:\n- High Knees: 3 sets of 6 reps\n- Push-ups: 3 sets of 15 reps\n- Jump Rope: 3 sets of 12 reps\nFriday:\n- Russian Twists: 3 sets of 20 reps\n- Walking: 3 sets of 15 reps\n- Jump Rope: 3 sets of 10 reps\nSaturday:\n- Rest Day\nSunday:\n- Rest Day', 'role': 'assistant'}]


In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model,device=0, tokenizer=tokenizer, max_new_tokens=512)


In [ ]:
prompt = pipe.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\nI am a 35-year-old male with a height of 161 cm and a weight of 50 kg. My primary goal is muscle gain, and I aim to reach a target weight of 55 kg. I am a beginner at the gym. Can you create a 7-day workout plan for me?<end_of_turn>\n<start_of_turn>model\nMonday:\n- Bicep Curls: 3 sets of 12 reps\n- Squats: 3 sets of 20 reps\n- Running: 3 sets of 8 reps\nTuesday:\n- Russian Twists: 3 sets of 12 reps\n- Shoulder Press: 3 sets of 12 reps\n- Jump Rope: 3 sets of 8 reps\nWednesday:\n- Rest Day\nThursday:\n- High Knees: 3 sets of 6 reps\n- Push-ups: 3 sets of 15 reps\n- Jump Rope: 3 sets of 12 reps\nFriday:\n- Russian Twists: 3 sets of 20 reps\n- Walking: 3 sets of 15 reps\n- Jump Rope: 3 sets of 10 reps\nSaturday:\n- Rest Day\nSunday:\n- Rest Day<end_of_turn>\n<start_of_turn>model\n'

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

**Additional Tips:**

- Start slowly and gradually increase weight and reps over time.
- Focus on compound exercises that work multiple muscle groups at once.
- Warm up before each workout with 5-10 minutes of light cardio and dynamic stretching.
- Cool down after each workout with static stretching.
- Eat a healthy diet rich in protein, carbohydrates, and healthy fats.
- Get enough sleep for optimal recovery.
- Stay hydrated by drinking plenty of water throughout the day.


#파인튜닝


In [ ]:
def generate_train_prompt(example):
    prompt_list = []
    for i in range(len(example['QUESTION'])):
        prompt_list.append(r"""<bos><start_of_turn>user

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['QUESTION'][i], example['ANSWER'][i]))
    return prompt_list

In [ ]:
train_data = dataset_dict['train']
#print(train_data)
print(generate_train_prompt(train_data[:1])[0])

<bos><start_of_turn>user

I am a 54-year-old male with a height of 189 cm and a weight of 63 kg. My primary goal is health maintenance, and I aim to reach a target weight of 93 kg. I am a intermediate at the gym. Can you create a 7-day workout plan for me?<end_of_turn>
<start_of_turn>model
Monday:
- Sit-ups: 3 sets of 6 reps
- Planks: 3 sets of 20 reps
- Walking: 3 sets of 12 reps
Tuesday:
- Running: 3 sets of 6 reps
- High Knees: 3 sets of 15 reps
- Kettlebell Swings: 3 sets of 8 reps
Wednesday:
- Rest Day
Thursday:
- Lunges: 3 sets of 8 reps
- High Knees: 3 sets of 15 reps
- Kettlebell Swings: 3 sets of 10 reps
Friday:
- High Knees: 3 sets of 6 reps
- Deadlifts: 3 sets of 15 reps
- Shoulder Press: 3 sets of 8 reps
Saturday:
- Rest Day
Sunday:
- Rest Day<end_of_turn><eos>


In [ ]:
from transformers import BitsAndBytesConfig
import bitsandbytes
from peft import LoraConfig, PeftModel
import torch

lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=90,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_train_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,1.004200
200,0.306500
300,0.191300
400,0.100700
500,0.062900
600,0.047000
700,0.039800
800,0.034200
900,0.031200
1000,0.030000


TrainOutput(global_step=3000, training_loss=0.07969607901573181, metrics={'train_runtime': 6145.1667, 'train_samples_per_second': 1.953, 'train_steps_per_second': 0.488, 'total_flos': 3.913005354054451e+16, 'train_loss': 0.07969607901573181, 'epoch': 8.333333333333334})

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
!ls -alh lora_adapter

total 29M
drwxr-xr-x 2 root root 4.0K Oct  3 06:05 .
drwxr-xr-x 1 root root 4.0K Oct  3 06:05 ..
-rw-r--r-- 1 root root  719 Oct  3 06:05 adapter_config.json
-rw-r--r-- 1 root root  29M Oct  3 06:05 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Oct  3 06:05 README.md


In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-diet-manager')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!ls -alh ./gemma-2b-it-diet-manager

total 4.7G
drwxr-xr-x 2 root root 4.0K Oct  3 06:07 .
drwxr-xr-x 1 root root 4.0K Oct  3 06:07 ..
-rw-r--r-- 1 root root  706 Oct  3 06:07 config.json
-rw-r--r-- 1 root root  132 Oct  3 06:07 generation_config.json
-rw-r--r-- 1 root root 4.7G Oct  3 06:07 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Oct  3 06:07 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Oct  3 06:07 model.safetensors.index.json


In [ ]:
huggingface_hub.login()

In [ ]:
model.push_to_hub("sawadikab/diet-manager")

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sawadikab/diet-manager/commit/8b6f4b61e5336b7ace837435f21e99e7fe635592', commit_message='Upload GemmaForCausalLM', commit_description='', oid='8b6f4b61e5336b7ace837435f21e99e7fe635592', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sawadikab/diet-manager', endpoint='https://huggingface.co', repo_type='model', repo_id='sawadikab/diet-manager'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("sawadikab/diet-manager")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sawadikab/diet-manager/commit/cbbe5d110b27e2c1667c3be7061f928e0c7d40b1', commit_message='Upload tokenizer', commit_description='', oid='cbbe5d110b27e2c1667c3be7061f928e0c7d40b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sawadikab/diet-manager', endpoint='https://huggingface.co', repo_type='model', repo_id='sawadikab/diet-manager'), pr_revision=None, pr_num=None)

#결과 확인

In [ ]:
new_model_name = "sawadikab/diet-manager"

new_model = AutoModelForCausalLM.from_pretrained(new_model_name)
new_tokenizer = AutoTokenizer.from_pretrained(new_model_name)


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

new_pipe = pipeline("text-generation", model=new_model,device=0, tokenizer=new_tokenizer, max_new_tokens=512)

message = dataset_dict['train'][0]['messages']

prompt = new_pipe.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = new_pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Приветствуем вас на cviковании! Мы chantierт вас 7-надесяной планировкой работы out. Оживите свои силы и достичь своих целей!
Monday:
- Wednesdayks: 3 sets of 8 reps
- Mountain Climbers: 3 sets of 20 reps
- Push-ups: 3 sets of 12 reps
Tuesday:
- Leg Press: 3 sets of 12 reps
- Bench Press: 3 sets of 6 reps
- High Knees: 3 sets of 15 reps
Wednesday:
- Rest Day
Thursday:
- Rest Day
Friday:
- Deadlifts: 3 sets of 10 reps
- Russian Twists: 3 sets of 12 reps
- Jumping Jacks: 3 sets of 6 reps
Saturday:
- Rest Day
Sunday:
- Rest Day


파인튜닝전:
**Additional Tips:**

- Start slowly and gradually increase weight and reps over time.
- Focus on compound exercises that work multiple muscle groups at once.
- Warm up before each workout with 5-10 minutes of light cardio and dynamic stretching.
- Cool down after each workout with static stretching.
- Eat a healthy diet rich in protein, carbohydrates, and healthy fats.
- Get enough sleep for optimal recovery.
- Stay hydrated by drinking plenty of water throughout the day.

파인튜닝 후:
Monday:
- Wednesdayks: 3 sets of 8 reps
- Mountain Climbers: 3 sets of 20 reps
- Push-ups: 3 sets of 12 reps
Tuesday:
- Leg Press: 3 sets of 12 reps
- Bench Press: 3 sets of 6 reps
- High Knees: 3 sets of 15 reps
Wednesday:
- Rest Day
Thursday:
- Rest Day
Friday:
- Deadlifts: 3 sets of 10 reps
- Russian Twists: 3 sets of 12 reps
- Jumping Jacks: 3 sets of 6 reps
Saturday:
- Rest Day
Sunday:
- Rest Day

